# ◆[Backtest](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.buy) with OANDA API ver.H4
##### 目的：過去約2年の米ドル4時間足にて、バックテストを行う。高SQNスコアを目指す。
###### Create Date：2020/02/14　Author：M.Hasegawa
### ────────────────────────────────────────────────────────────────

#### 【table of contents】
- 0.[**Import module**](#Import_module)
- 1.[**Import data**](#Import_data)
- 2.[**Strategy**](#Strategy)
- 3.[**Backtest**](#Backtest)
- 4.[**Optimisation**](#Optimisation)

- ref:https://kernc.github.io/backtesting.py/doc/examples/Multiple%20Time%20Frames.html
- ref:https://saidataisei.hatenablog.com/entry/2019/10/13/003622
- ref:http://mmorley.hatenablog.com/entry/fx_backtesting01

## 0. Import module<a id='Import_module'></a>
- pip install backtesting
- pip install mpl_finance
- pip install oandapyV20
- pip install git+https://github.com/oanda/oandapy.git
- pip install TA_Lib-0.4.17-cp37-cp37m-win_amd64.whl

###### TA-LIB
- ref: https://qiita.com/ConnieWild/items/cb50f36425a683c914d2
- ref: https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib

In [1]:
%matplotlib inline
import oandapy
import pytz
import configparser
import pandas as pd
import numpy  as np
import talib as ta
from datetime         import datetime
from oandapyV20       import API
from backtesting      import Backtest, Strategy
from backtesting.lib  import crossover
from backtesting.lib  import resample_apply
from backtesting.test import SMA, GOOG
from pandas.core      import resample

config = configparser.ConfigParser()
config.read('./config_OANDA.txt')
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']

import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# Conv Japan Time
# ============================================================================
def iso_to_jp(iso):
    date = None
    try:
        date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
        date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
            date = date.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return date

# ============================================================================
# Conv Format
# ============================================================================
def date_to_str(date):
    if date is None:
        return ''
    return date.strftime('%Y/%m/%d %H:%M:%S')

# ============================================================================
# SMA
# ============================================================================
def SMA(array, n):
    return pd.Series(array).rolling(n).mean()

# ============================================================================
# EMA
# ============================================================================
def EMA(array, n):
    return list(pd.Series(array).ewm(span=n).mean())

# ============================================================================
# RSI
# ============================================================================
def RSI(array, n):
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

# ============================================================================
# RCI:http://lowcost-greatidea.jp/technicalanalysis/rci/
# ============================================================================
def RCI(close, period):
    rank_period = np.arange(period, 0, -1)
    length = len(close)
    rci = np.zeros(length)
    
    for i in range(length):
        if i < period - 1:
            rci[i] = 0
        else :
            rank_price = pd.Series(close)[i - period + 1: i + 1].rank(method='min', ascending = False).values
            rci[i] = np.int32((1 - (6 * sum((rank_period - rank_price)**2)) / (period**3 - period)) * 100)
    return rci

# ============================================================================
# BB2
# ============================================================================
def bb2(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper2, middle, lower2 = ta.BBANDS(gain.close, n,2,2,0)
    gain['bb_upper'] = upper2
    gain['bb_lower'] = lower2
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# BB3
# ============================================================================
def bb3(array, n):
    gain=pd.DataFrame(array)
    gain.columns=['close']
    upper3, middle, lower3 = ta.BBANDS(gain.close, n,3,3,0)
    gain['bb_upper'] = upper3
    gain['bb_lower'] = lower3
    return gain['bb_upper'],gain['bb_lower']

# ============================================================================
# ADX
# ============================================================================
def adx(array,n):
    gain=pd.DataFrame(array)
    gain=gain.T
    gain.columns=['close','high','low']
    gain['adx'] = ta.ADX(gain['high'],gain['low'],gain['close'],n)
    return gain['adx']

C:\Users\arsma\Anaconda3\lib\site-packages\backtesting\_plotting.py:37: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# API呼出
oa = oandapy.API(environment="live", access_token=api_key)

def df_init(data):
    df = pd.DataFrame(data["candles"])
    df['time'] = df['time'].apply(lambda x: iso_to_jp(x))   # 日本時間に変換
    df['time'] = df['time'].apply(lambda x: date_to_str(x)) # 形式変換（文字列型）
    df["time"] = pd.to_datetime(df["time"])                  # 型変換
    df = df[["time","openAsk","highAsk","lowAsk","closeAsk"]].copy()
    df = df.rename(columns={"openAsk":"Open","highAsk":"High","lowAsk":"Low","closeAsk":"Close"})
    df = df.set_index("time")
    return df

data = oa.get_history(instrument="USD_JPY", granularity='H4', count=200)
df_200 = df_init(data)

In [3]:
class RCIStrategy(Strategy):

    n1 = 21; n2 = 75
    rci_s = 9; rci_m = 24; rci_l = 52
    rsi_m = 14
    ema_s = 10
    
    def init(self):
        # SMA
        self.sma_s = self.I(SMA, self.data.Close, self.n1)
        self.sma_l = self.I(SMA, self.data.Close, self.n2)
        # RSI
        self.rsi_m = self.I(RSI, self.data.Close, self.rsi_m)
        # BB
        self.bb_u2,self.bb_l2 = self.I(bb2,self.data.Close,  self.n1)
        self.bb_u3,self.bb_l3 = self.I(bb3,self.data.Close,  self.n1)
        # RCI
        self.rci_s = self.I(RCI, self.data.Close, self.rci_s)
        self.rci_m = self.I(RCI, self.data.Close, self.rci_m)
        self.rci_l = self.I(RCI, self.data.Close, self.rci_l)
        # EMA
        self.ema_s = self.I(EMA, self.data.Close, self.ema_s)
        
    def next(self):
        
        price = self.data.Close[-1] # 前回の終値
        rci_s = self.rci_s[-1]
        rci_m = self.rci_m[-1]
        rci_l = self.rci_l[-1]
        
        # *******************************************************************************
        # ■ ロング
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # -----------------------------------------------------------------------
             # ▼ Entry Long ※理想は3重底
             # -----------------------------------------------------------------------
             or (rci_s < -98 and rci_m < -90)
             or (rci_s < -82 and rci_m < 0 and rci_l > -71)
             or (rci_s < -63 and rci_m < -35 and rci_l > -75) 
            )):
            self.buy(sl=.982 * price) # ※ストップ：-200pips
            
        elif (self.position.is_short and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Switch_Long ※ 保有中のショートを閉じてロングする
               # -----------------------------------------------------------------------
               or (rci_s < -87) 
               or (rci_s < -85 and rci_m > 75)
               or (rci_s < -75 and rci_l > 90)
               or (rci_s < -55 and rci_m > 84 and rci_l > 67)
             )):
            self.buy(sl=.982 * price) # ※ストップ：-200pips
            
        elif (self.position.is_long and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Exit Long ※理想は3天井
               # -----------------------------------------------------------------------
               or (rci_m > 88)
               or (rci_l > 75)
               or (rci_s > 61 and rci_l > 27)
              )):
            self.position.close() 
            
        # *******************************************************************************
        # ■ ショート
        # *******************************************************************************
        if (not self.position and ((1==0) 
             # -----------------------------------------------------------------------
             # ▼ Entry Short ※理想は3天井
             # -----------------------------------------------------------------------
             or (rci_s > 72 and rci_m > 36) # EntryShort1 修正
             or (rci_s > 70 and rci_l > 61)
             or (rci_s > 85 and rci_m < -50 and rci_l < -90) # 戻り売りの可能性
            )):
            self.sell(sl=1.012 * price) # ※ストップ：-130pips
            
        elif (self.position.is_long and ((1==0) 
               # -----------------------------------------------------------------------
               # ▼ Switch_Short ※ 保有中のロングを閉じてショートする
               # -----------------------------------------------------------------------
               or (rci_s > 95 and rci_m > 53)
               or (rci_s > -10 and crossover(self.sma_l, self.sma_s)) # デッドクロス(売りシグナル：短期下、長期上)
             )):
            self.sell(sl=1.012 * price) # ※ストップ：-130pips
            
        elif (self.position.is_short and ((1==0)
               # -----------------------------------------------------------------------
               # ▼ショートのExitルール(理想は3重底)
               # -----------------------------------------------------------------------
               or (rci_s < -91)
               or (rci_m < -86)
               or (rci_s < -87 and rci_m < -15)
               or (rci_s < -70 and rci_m < -52 and rci_l < -26)
              )):
            self.position.close()

In [4]:
bt = Backtest(df_200,
              RCIStrategy,
              cash=1000,  # 所持金1000ドル(=約10万)
              commission=0.0002, # 取引手数料（為替価格に対する倍率で指定、為替価格100円でcommission=0.0002なら0.02円）
              trade_on_close=True # True：現在の終値に関してエントリー／ False：次の始値にエントリー
             )
out=bt.run()

xls_df = df_200
xls_df['RCI_s'] = RCI(xls_df.Close,9)
xls_df['RCI_m'] = RCI(xls_df.Close,24)
xls_df['RCI_l'] = RCI(xls_df.Close,52)
xls_df['SMA_s'] = SMA(xls_df.Close,21)
xls_df['SMA_l'] = SMA(xls_df.Close,75)


out_df = pd.concat([out._trade_data, xls_df], axis=1)
out_df = out_df.fillna({'SMA_s': 0.0})
out_df = out_df.fillna({'SMA_l': 0.0})

out_df = out_df[['Equity','Entry Price','Exit Price','Returns','RCI_s','RCI_m','RCI_l','SMA_s','SMA_l']]
out_df = out_df.tail(20)

#out_df.to_csv("./AutoOutput.csv")
display(out_df.tail(20))

,Equity,Entry Price,Exit Price,Returns,RCI_s,RCI_m,RCI_l,SMA_s,SMA_l
time,,,,,,,,,
2020-02-11 11:00:00,1017.074609,NaN,NaN,NaN,22.0,-16.0,75.0,109.839286,109.285613
2020-02-11 15:00:00,1017.361566,NaN,NaN,NaN,54.0,-23.0,77.0,109.839714,109.290493
2020-02-11 19:00:00,1018.129868,NaN,NaN,NaN,67.0,-32.0,77.0,109.837333,109.294640
2020-02-11 23:00:00,1017.833655,NaN,NaN,NaN,68.0,-43.0,77.0,109.832810,109.298213
2020-02-12 03:00:00,1018.065071,NaN,NaN,NaN,50.0,-43.0,76.0,109.829905,109.301133
2020-02-12 07:00:00,1017.315282,NaN,NaN,NaN,43.0,-37.0,76.0,109.829333,109.305600
2020-02-12 11:00:00,1017.444876,NaN,NaN,NaN,23.0,-24.0,76.0,109.823857,109.312773
2020-02-12 15:00:00,1016.185971,NaN,NaN,NaN,23.0,-10.0,76.0,109.823619,109.322080
2020-02-12 19:00:00,1016.278537,NaN,NaN,NaN,41.0,6.0,77.0,109.826952,109.335467
